In [1]:
from get_test import get_csv_names

In [2]:
ecg_dir='/home/ec2-user/SageMaker/datasets/Physionet2016_a/new_split_basedon_rawpcg/raw/ecg/'
pcg_dir='/home/ec2-user/SageMaker/datasets/Physionet2016_a/new_split_basedon_rawpcg/raw/pcg/'


a=get_csv_names('other_records.csv', ecg_dir, pcg_dir) # 'other_records.csv/ test_records.csv'
b=get_csv_names('test_records.csv', ecg_dir, pcg_dir) # 'other_records.csv/ test_records.csv'

In [3]:
print(a[:5])
print(b[:5])

['a0001_1550_A.tiff', 'a0001_17462_A.tiff', 'a0001_25037_A.tiff', 'a0001_32256_A.tiff', 'a0001_40080_A.tiff']
['a0126_15998_A.tiff', 'a0126_23079_A.tiff', 'a0126_273_A.tiff', 'a0126_30108_A.tiff', 'a0126_37175_A.tiff']


In [5]:
len(a)

3067

In [1]:
import cv2 as cv
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
import random
from get_test import get_csv_names



In [10]:

#Default Settings
Dataset_path = ''
Ignore_Blanks = True
Read_Type = 0 


images = []

#Default Picture Crop Points
s_x = 82;
e_x = 544;
s_y = 43;
e_y = 315;

size_x = e_x - s_x ;
size_y = e_y - s_y;

n = 0 
n_cat = 3;

print(size_y,size_x)

def is_blank(x):
    img = cv.imread(Dataset_path + x, 0)
    
    if np.sum(img[150:200,350:400])==637500:
        return True
    else:
        return False

def set_ignore_blanks(x):
    global Ignore_Blanks
    Ignore_Blanks = x

def set_dataset_path(path,readtype,ignore_folder=None):
    global Read_Type
    Read_Type = readtype
    
    global Dataset_path
    Dataset_path = path
    
    global images
    form = '.tiff'
    images = sorted(os.listdir(Dataset_path))
    images = [image for image in images if form in image]
    print(" 💡 => Total images ("+ form +") found ",len(images))
    
    if(ignore_folder!=None):
        images = [image for image in images if ignore_folder not in image]
        print(" ⚡️ => After ignoring folder loaded ",len(images))
    
    if(Ignore_Blanks):
        images = [image for image in images if is_blank(image)==0]
        print(" ⚡️ => After ignoring blanks loaded ",len(images))

    global n
    n = len(images)
    
    cat_2_count = 0
    
    for i in range(0,n):
        if images[i][-6]=='N':
            cat_2_count = cat_2_count + 1
        elif images[i][-6]=='A' or images[i][-6]=='O': #other or Abnormal
            cat_2_count = cat_2_count + 1
            
    n = cat_2_count
    
    return True

def show_i(x):
    img = cv.imread(Dataset_path + images[x], 0)
    plt.figure(figsize=(10,8), dpi=100)
    
    plt.imshow(img[s_y:e_y,s_x:e_x])
    plt.xticks([])
    plt.yticks([])
    plt.box(False)
    plt.show()
    
def show_img(x):
    plt.figure(figsize=(10,8), dpi=100)
    
    plt.imshow(x)
    plt.xticks([])
    plt.yticks([])
    plt.box(False)
    plt.show()
    
X = None
Y = None
Y = None



272 462


In [52]:


def load_hybrid_data(high_res,leave_test= False , test_set=None):
    global s_x
    global e_x
    global s_y
    global e_y

    height = e_y - s_y;
    width  = e_x - s_x
    
    ecg_path  = Dataset_path[:-4] + 'ecg/'
    ecg_files = os.listdir(ecg_path)
    
    test_records =get_csv_names('test_records.csv', ecg_dir, Dataset_path) # 'other_records.csv/ test_records.csv'

    global images
    images = [image for image in images if image in ecg_files]
    
    
    if test_set=='testset':
        images = [image for image in images if image in test_records]
    else:
        images = [image for image in images if image not in test_records]
    
    return images
    
    '''
    global n
    n = len(images)
    
    m = Read_Type*3 + int(Read_Type==0)
    e = Read_Type*6 + int(Read_Type==0)*2
    
    X = np.zeros(shape=(n,height,width,e),dtype=np.uint8)
    Y = np.zeros(shape=(n))
    Y = Y.astype('int8') 
    
    for i in range(0,n):
        
        im1 = cv.imread(Dataset_path + images[i], Read_Type)
        im2 = cv.imread(ecg_path     + images[i], Read_Type)
        
        im1 = im1[s_y:e_y,s_x:e_x]
        im2 = im2[s_y:e_y,s_x:e_x]
        
        im1 = im1.reshape((height,width,m))
        im2 = im2.reshape((height,width,m))
        
        X[i,:,:,0:m] = im1
        X[i,:,:,m:e] = im2

        if images[i][-6]=='N':
            Y[i] = 0
        else:
            Y[i] = 1

        if(i%1000==0):
            print(i)       

    print(str(n)+ " Images loaded across " + '2' + " Categories")  
    
    return n,X,Y
    '''


def load_data(train_percentage):
    X = None
    Y = None
    
    if(Read_Type==1):
        X = np.zeros(shape=(n,272,462,3),dtype=np.uint8)
        Y = np.zeros(shape=(n))
        Y = Y.astype('int8') 
    elif(Read_Type==0):
        X = np.zeros(shape=(n,272,462),dtype=np.uint8)
        Y = np.zeros(shape=(n))
        Y = Y.astype('int8')
            
    if(len(Dataset_path)==0):
        print("Dataset path not set")
        return 
    
    for i in range(0,n):
        img = cv.imread(Dataset_path + images[i], Read_Type)
        
        
        X[i]= img[s_y:e_y,s_x:e_x]
        
        
        

        if images[i][-6]=='N':
            Y[i] = 0
        else:
            Y[i] = 1

        if(i%1000==0):
            print(i)

    print(str(n)+ " Images loaded across " + '2' + " Categories")  
    
    return n,X,Y

In [53]:
NEPTUNE_API_TOKEN="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiNjBlODllY2ItMDcyZC00OGNmLThjZWYtZjM2MjE4Y2M5ZDM0In0="
PARAMS = {'ignore_blanks' : True,
          'epochs' : 8,
         'validation_split':0.2,
         'batch_size' : 20,
         'train_test_split':0.8,
         'dense_layers':80,
          'dropout':0.1,
         'dataset_path': '/home/ec2-user/SageMaker/datasets/Physionet2016_a/new_split_basedon_rawpcg/raw/pcg/',
         'image_type' : 'rgb',
         'im_type':6,
         'Read_Type':1,
          'data_balanced':False,
         }

In [58]:
p = 0.8
set_ignore_blanks(True)
set_dataset_path(PARAMS['dataset_path'],PARAMS['Read_Type'])


names = load_hybrid_data(False ,leave_test= True)



 💡 => Total images (.tiff) found  3496
 ⚡️ => After ignoring blanks loaded  3350


In [59]:
names ==sorted(b)


False

In [63]:
xx = [aa for aa in a if aa not in names]

In [64]:
len(xx)

146

In [49]:
sorted(b)==names

True

In [152]:
def select_records(x):
    records = np.array(pd.read_csv('REFERENCE.csv'))[:,]  

    abnormal_records = []
    normal_records   = []

    for i in range(0,len(records)):
        if(records[i][1]==-1):
            normal_records.append(records[i][0])
        else:
            abnormal_records.append(records[i][0])

    random.seed(x)
    abnormal_records_selected = random.sample(abnormal_records, k=len(normal_records))
    
    return normal_records,abnormal_records_selected



In [153]:
n_r,a_r = select_records(120)

In [154]:
images = sorted(os.listdir('/home/ec2-user/SageMaker/datasets/Physionet2016_a/new_split_basedon_rawpcg/raw/pcg/'))
images = [image for image in images if 'tiff' in image]

In [155]:
len(c_images)

2001

In [156]:
len(images)

3496

In [157]:
count = 0 



c_images = images[:]

for i in range(0,len(images)):
    rec = images[i][0:5]
    if(not(rec in n_r or rec in a_r)):
        c_images.remove(images[i])
        count = count + 1
    
        
print(count)
print(len(c_images))

1495
2001


In [114]:
len(abnormal_records_selected)

117

In [115]:
len(set(abnormal_records_selected))

117

In [116]:
sorted(abnormal_records_selected)

['a0002',
 'a0003',
 'a0010',
 'a0018',
 'a0021',
 'a0022',
 'a0031',
 'a0033',
 'a0039',
 'a0041',
 'a0048',
 'a0051',
 'a0052',
 'a0056',
 'a0057',
 'a0062',
 'a0064',
 'a0066',
 'a0072',
 'a0075',
 'a0076',
 'a0077',
 'a0078',
 'a0083',
 'a0084',
 'a0090',
 'a0095',
 'a0096',
 'a0100',
 'a0101',
 'a0107',
 'a0110',
 'a0117',
 'a0119',
 'a0120',
 'a0121',
 'a0122',
 'a0126',
 'a0128',
 'a0130',
 'a0132',
 'a0135',
 'a0137',
 'a0138',
 'a0143',
 'a0145',
 'a0149',
 'a0150',
 'a0152',
 'a0157',
 'a0160',
 'a0167',
 'a0170',
 'a0172',
 'a0175',
 'a0177',
 'a0180',
 'a0188',
 'a0190',
 'a0194',
 'a0197',
 'a0198',
 'a0200',
 'a0203',
 'a0206',
 'a0207',
 'a0213',
 'a0215',
 'a0216',
 'a0217',
 'a0218',
 'a0219',
 'a0225',
 'a0226',
 'a0230',
 'a0232',
 'a0239',
 'a0247',
 'a0252',
 'a0255',
 'a0258',
 'a0260',
 'a0263',
 'a0265',
 'a0269',
 'a0273',
 'a0276',
 'a0277',
 'a0280',
 'a0284',
 'a0292',
 'a0303',
 'a0306',
 'a0307',
 'a0308',
 'a0312',
 'a0315',
 'a0316',
 'a0322',
 'a0326',
